In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
%matplotlib inline

In [3]:
import gym
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
obs = env.reset()
num_params = len(obs)

In [33]:
def get_ranks(x):
    assert x.ndim == 1
    ranks = np.empty(len(X), dtype=int)
    ranks[x.argsort()] = np.arange(len(x))
    return ranks

def norm_ranks(x):
    ranks = get_ranks(x.ravel()).reshape(x.shape).astype(np.float32)
    ranks /= (x.size - 1)
    ranks -= 0.5
    return ranks

def get_w_decay(w_decay, params_list):
    params_array = np.array(params_list)
    return -w_decay * np.mean(params_array*params_array, axis=1)
    

In [34]:
class Optimizer(object):
    def __init__(self, pi, epsilon=1e-08):
        self.pi = pi
        self.dim = pi.num_params
        self.epsilon = epsilon
        self.t = 0

    def update(self, globalg):
        self.t += 1
        step = self._compute_step(globalg)
        theta = self.pi.mu
        ratio = np.linalg.norm(step) / (np.linalg.norm(theta) + self.epsilon)
        self.pi.mu = theta + step
        return ratio

    def _compute_step(self, globalg):
        raise NotImplementedError


class BasicSGD(Optimizer):
    def __init__(self, pi, stepsize):
        Optimizer.__init__(self, pi)
        self.stepsize = stepsize

    def _compute_step(self, globalg):
        step = -self.stepsize * globalg
        return step

class SGD(Optimizer):
    def __init__(self, pi, stepsize, momentum=0.9):
        Optimizer.__init__(self, pi)
        self.v = np.zeros(self.dim, dtype=np.float32)
        self.stepsize, self.momentum = stepsize, momentum

    def _compute_step(self, globalg):
        self.v = self.momentum * self.v + (1. - self.momentum) * globalg
        step = -self.stepsize * self.v
        return step


class Adam(Optimizer):
    def __init__(self, pi, stepsize, beta1=0.99, beta2=0.999):
        Optimizer.__init__(self, pi)
        self.stepsize = stepsize
        self.beta1 = beta1
        self.beta2 = beta2
        self.m = np.zeros(self.dim, dtype=np.float32)
        self.v = np.zeros(self.dim, dtype=np.float32)

    def _compute_step(self, globalg):
        a = self.stepsize * np.sqrt(1 - self.beta2 ** self.t) / (1 - self.beta1 ** self.t)
        self.m = self.beta1 * self.m + (1 - self.beta1) * globalg
        self.v = self.beta2 * self.v + (1 - self.beta2) * (globalg * globalg)
        step = -a * self.m / (np.sqrt(self.v) + self.epsilon)
        return step

(4, 2)

In [4]:
def evaluate(W):
    X = env.reset()
    for t in range(1,201):
        action = 0 if W@X < 0 else 1
        X, reward, done, _ = env.step(action)
        if done:
            return t
    return t

In [41]:
x = 1e-4;x+1

1.0001

In [ ]:
class openES:
    """Basic version of OpenAI ES"""
    def __init__(self, 
                 num_params, 
                 sig_init=0.1, 
                 sig_decay=0.999, 
                 sig_lim=0.01, 
                 w_decay=1e-2,  
                 popsize=256, 
                 lr=1e-2, 
                 lr_decay=0.9999, 
                 lr_lim=1e-3, 
                 antithetic=False, 
                 rank_fit=True, 
                 forget_best=True):

In [11]:
MY_REQUIRED_FITNESS = 199

In [12]:
solver = SimpleGA(num_params)
e = 0
while e < 100:

    # ask the ES to give us a set of candidate solutions
    solutions = solver.ask()

    # create an array to hold the fitness results.
    fitness_list = np.zeros(solver.popsize)

    # evaluate the fitness for each given solution.
    for i in range(solver.popsize):
        fitness_list[i] = evaluate(solutions[i])

    # give list of fitness results back to ES
    solver.tell(fitness_list)

    # get best parameter, fitness from ES
    best_solution, best_fitness_ever, best_fitness_current, sigma = solver.result()
    e += 1
    print(e, best_fitness_ever, best_fitness_current)
    #print (best_fitness)
    if best_fitness_ever > MY_REQUIRED_FITNESS:
        break

1 200.0 200.0
